In [2]:
import pandas as pd

import google.generativeai as genai

from IPython.display import Markdown, display

from markdown_it import MarkdownIt
from markdown_it import utils


import time
import json
import os

from dotenv import load_dotenv

load_dotenv()

True

In [3]:
# Connect to Gemini
genai.configure(api_key=os.environ['GOOGLE_API_KEY'])

model = genai.GenerativeModel("gemini-1.5-flash")

In [4]:
# Import data for retrieval docs
abilities_entries = pd.read_json("Cleaned-Jsons/fandom_abilities.json")
maps_entries = pd.read_json("Cleaned-Jsons/fandom_maps.json")
players_entries = pd.read_json("Cleaned-Jsons/fandom_players.json") # change player id to player gamer tag
teams_entries = pd.read_json("Cleaned-Jsons/fandom_teams.json")
tournaments_entries = pd.read_json("Cleaned-Jsons/fandom_tournaments.json")
weapons_entries = pd.read_json("Cleaned-Jsons/fandom_weapons.json")

In [5]:
# Change abilities columns
abilities_entries.rename(columns={'title': 'ability name'}, inplace=True)

# Change maps columns
maps_entries.rename(columns={'title': 'map name'}, inplace=True)

# Change players columns
players_entries.rename(columns={'id': 'player'}, inplace=True)

# Change teams columns
teams_entries.rename(columns={'name': 'team name'}, inplace=True)

# Change tournaments columns
tournaments_entries.rename(columns={'name': 'tournament name', 'sdate': 'start date'}, inplace=True)

# Change weapons columns
weapons_entries.rename(columns={'title': 'weapon'}, inplace=True)

In [6]:
# Consolidate all entries

# Part 1
all_entries = [abilities_entries,maps_entries,teams_entries,tournaments_entries,weapons_entries]

# Part 2
#all_entries = [players_entries]

In [7]:
total_rows = sum(len(df) for df in all_entries)
total_rows

583

In [8]:
# Stored consolidated string entries
docs = []

# Convert dataframe rows into strings
def rowToString(row):

    row_str = []

    for col_name, col_value in row.items():

            row_str.append(f"{col_name}: {col_value}")

    return ' | '.join(row_str)

# Convert all and add to corpus
for topic in all_entries:

      # Convert to strings
      converted_rows = (topic.apply(rowToString, axis=1)).tolist()
      docs.append(converted_rows)

docs[0][0]

"ability name: Curveball | summary:  | stats: {'20px]] Equip time': '0.8 seconds', '20px]] Unequip time': '0.48 seconds', '20px]] Windup time': '0.5 seconds', '20px]] Max blind duration': '1.5 seconds'} | description: Throw a curving flare orb that detonates to blind any player who sees it. | agent: {{ai|Phoenix}} | friendlyfire: Yes | creds: 250 | uses: 2 | fulleffects:  | health:  | restock:  | points:  | mapobjects:  | function: [[Blind#Flash|Flash]] | uses_per_round:  | credits: "

In [12]:
# Create prompt
def prompt(doc):
    prompt = f"""
        For the data provided, please summarize the information and drop empty items that are not useful to make it easy to parse by a language model. Return in JSON records row format. No newlines. Do not include markdown: {doc} 
        """
    return prompt

In [13]:
# Get test result
query = prompt(docs[0][13])

response = model.generate_content(prompt(query))
response.text
# Prettify Response
#display(Markdown(response.text))

'{"ability_name": "Updraft", "summary": "", "stats": {"Duration": "0.6 seconds", "Dash distance": "5 meters"}, "description": "Propel yourself high into the air.", "agent": "Jett", "friendlyfire": "", "creds": 150, "uses": 1, "fulleffects": "", "health": "", "restock": "", "points": "", "mapobjects": "", "function": "Dash", "uses_per_round": "", "credits": ""} \n'

In [14]:
json.loads(response.text)

{'ability_name': 'Updraft',
 'summary': '',
 'stats': {'Duration': '0.6 seconds', 'Dash distance': '5 meters'},
 'description': 'Propel yourself high into the air.',
 'agent': 'Jett',
 'friendlyfire': '',
 'creds': 150,
 'uses': 1,
 'fulleffects': '',
 'health': '',
 'restock': '',
 'points': '',
 'mapobjects': '',
 'function': 'Dash',
 'uses_per_round': '',
 'credits': ''}

In [15]:
# Iterate through each entry for each topic
cleaned_docs = []

flagged_docs = []

for topic_docs in docs:

      cleaned_topic = []

      flagged_topic = []

      for entry in topic_docs:

            # Check if flagged
            try:

                  response = model.generate_content(prompt(entry)).text
            
                  cleaned_topic.append(response)

                  print(response)

                  

            except ValueError as e:

                  flagged_topic.append(entry)
                  print(e)

            time.sleep(5)     # Limit rate per entry to be under rate limit
            



      cleaned_docs.append(cleaned_topic)
      flagged_docs.append(flagged_topic)

{"ability_name": "Curveball", "summary": "Throw a curving flare orb that detonates to blind any player who sees it.", "stats": {"20px]] Equip time": "0.8 seconds", "20px]] Unequip time": "0.48 seconds", "20px]] Windup time": "0.5 seconds", "20px]] Max blind duration": "1.5 seconds"}, "agent": "Phoenix", "friendlyfire": "Yes", "creds": 250, "uses": 2, "function": "Flash"} 

{"ability_name": "Slow Orb", "summary": "Slow Orb is a Crowd Control ability that Sage must equip before casting. Upon casting she throws forward a slow orb that detonates on contact with horizontal terrain, covering a surrounding area in radianite crystals. Any agents who come into contact with these crystals are slowed and give off an audio cue whenever they run or jump across the zone. Once it expires, the zone will rapidly dissipate.", "stats": {"Equip time": "0.5 seconds", "Unequip time": "0.4 seconds", "Windup time": "0.5 seconds", "Slow amount": "50%", "Duration": "7 seconds"}, "description": "Throw a slowing 

In [16]:
for doc in flagged_docs:
    print(doc)

["ability name: Updraft | summary:  | stats: {'20px]] Duration': '0.6 seconds', '20px]] Dash distance': '5 meters'} | description: Propel yourself high into the air. | agent: {{ai|Jett}} | friendlyfire:  | creds: 150 | uses: 1 | fulleffects:  | health:  | restock:  | points:  | mapobjects:  | function: [[Mobility#Dash|Dash]] | uses_per_round:  | credits: ", 'ability name: Nanoswarm | summary:  | stats: {\'20px]] Equip time\': \'0.74 seconds\', \'20px]] Unequip time\': \'0.7 seconds\', \'20px]] Buy Phase pickup range\': \'4 meters\', \'20px]] Detection radius\': \'5.25 meters\', \'rowspan="3"\': \'20px]] Activation windup time\', \'0.5-second windup\': \'No*\', \'0.5-second damage delay\': \'Yes<sup/>Game files\', \'20px]] Radius\': \'4.5 meters\', \'20px]] Damage\': \'1 per tick<br>Total 180<br>x0.5 multiplier to objects\', \'rowspan="2"\': \'20px]] Ticks/Tick Rate\', \'Total 180 ticks\': \'Yes<sup/>[b]\', \'20px]] Duration\': \'4 seconds\'} | description: Throw a grenade that goes cov

In [17]:
# Save formatted and cleaned entries to JSON

# Part 1 Cleaned [abilities_entries,maps_entries,teams_entries,tournaments_entries,weapons_entries]


#maps_entries 
#cleaned_docs[1].to_json("Cleaned-Jsons/fandom_maps_cleaned.json", orient="records")

with open("Cleaned-Jsons/fandom_abilities_cleaned.txt", 'w', encoding='utf-8') as file:
    for entry in cleaned_docs[0]:
        file.write(entry + '\n')

#abilities_entries
#cleaned_docs[0].to_json("Cleaned-Jsons/fandom_abilities_cleaned.json", orient="records")

with open("Cleaned-Jsons/fandom_maps_cleaned.txt", 'w', encoding='utf-8') as file:
    for entry in cleaned_docs[1]:
        file.write(entry + '\n')

#teams_entries 
#cleaned_docs[3].to_json("Cleaned-Jsons/fandom_teams_cleaned.json", orient="records")

with open("Cleaned-Jsons/fandom_teams_cleaned.txt", 'w', encoding='utf-8') as file:
    for entry in cleaned_docs[2]:
        file.write(entry + '\n')

#tournaments_entries 
#cleaned_docs[4].to_json("Cleaned-Jsons/fandom_tournaments_cleaned.json", orient="records")

with open("Cleaned-Jsons/fandom_tournaments_cleaned.txt", 'w', encoding='utf-8') as file:
    for entry in cleaned_docs[3]:
        file.write(entry + '\n')

#weapons_entries 
#cleaned_docs[5].to_json("Cleaned-Jsons/fandom_weapons_cleaned.json", orient="records")

with open("Cleaned-Jsons/fandom_weapons_cleaned.txt", 'w', encoding='utf-8') as file:
    for entry in cleaned_docs[4]:
        file.write(entry + '\n')





# Part 2 Cleaned
#players_entries 
"""
cleaned_docs[2].to_json("Cleaned-Jsons/fandom_players_cleaned.json", orient="records")

with open("Cleaned-Jsons/fandom_players_cleaned.json", 'w') as file:
    for entry in cleaned_docs[0]:
        file.write(entry + '\n')

"""

'\ncleaned_docs[2].to_json("Cleaned-Jsons/fandom_players_cleaned.json", orient="records")\n\nwith open("Cleaned-Jsons/fandom_players_cleaned.json", \'w\') as file:\n    for entry in cleaned_docs[0]:\n        file.write(entry + \'\n\')\n\n'

In [18]:
count = 0
for doc in flagged_docs:
    if doc != []:
        count += 1

count

2

In [19]:
second_flag = []

for doc in flagged_docs:

            # Check if flagged
            try:

                  if doc == []:
                        print("---")
                  else:
                              
                        
                        response = model.generate_content(prompt(doc)).text
                  
                        print(response)

                  

            except ValueError as e:

                  second_flag.append(doc)
                  print(e)
                  print("Problem:\n===")
                  print(doc)
                  print("\n\n")

            time.sleep(5)     # Limit rate per entry to be under rate limit

Invalid operation: The `response.text` quick accessor requires the response to contain a valid `Part`, but none were returned. Please check the `candidate.safety_ratings` to determine if the response was blocked.
Problem:
===
["ability name: Updraft | summary:  | stats: {'20px]] Duration': '0.6 seconds', '20px]] Dash distance': '5 meters'} | description: Propel yourself high into the air. | agent: {{ai|Jett}} | friendlyfire:  | creds: 150 | uses: 1 | fulleffects:  | health:  | restock:  | points:  | mapobjects:  | function: [[Mobility#Dash|Dash]] | uses_per_round:  | credits: ", 'ability name: Nanoswarm | summary:  | stats: {\'20px]] Equip time\': \'0.74 seconds\', \'20px]] Unequip time\': \'0.7 seconds\', \'20px]] Buy Phase pickup range\': \'4 meters\', \'20px]] Detection radius\': \'5.25 meters\', \'rowspan="3"\': \'20px]] Activation windup time\', \'0.5-second windup\': \'No*\', \'0.5-second damage delay\': \'Yes<sup/>Game files\', \'20px]] Radius\': \'4.5 meters\', \'20px]] Damage\

In [20]:
len(second_flag)

1